In [42]:
import pandas as pd
import numpy as np
import seaborn as sns


   # Creating Classes

   **Road data handling class**

In [2]:
class Road_Features():
    def __init__(self):
        self.Signal_Handling()
  
        
    
        self.Speed_Handling()
        self.Camera_Handling()
        self.Incidents_Handling()
        self.Sign_Handling()
        self.Volume_Handling()
        

    def Speed_Handling(self):
        self.speed_limit=pd.read_csv('Speed_Limits.csv')
        
        self.speed_limit.rename(columns={'STREET_NAME':'Location','SPEED':'Speed','multiline':'Coordinates','CREATED_DT':'Date'},inplace=True)
        coord=self.speed_limit['Coordinates'].str.split(expand=True)
        
        for i in range(len(coord.columns)):
            if(i>0):
                coord[i]= coord[i].str.strip(')(,')
        coord.drop(columns=0,inplace=True)
        self.speed_limit['Date']=pd.to_datetime(self.speed_limit['Date'])
        self.speed_limit['Year']=pd.DatetimeIndex(self.speed_limit['Date']).year
        self.speed_limit=self.speed_limit[['Year','Speed']]
        self.speed_limit=pd.merge(self.speed_limit,coord,left_index=True,right_index=True)
        self.speed_limit=self.speed_limit.loc[self.speed_limit['Year']<=2018]
    def Camera_Handling(self):
        self.cameras=pd.read_csv('Traffic_Camera_Locations.csv')
        self.cameras=self.cameras[['longitude','latitude']]
#         print(self.cameras)
    def Incidents_Handling(self):
        self.incidents=pd.read_csv('Traffic_Incidents.csv')
        self.incidents.rename(columns={'INCIDENT INFO':'Location','START_DT':'Date'},inplace=True)
        self.incidents['Date']=pd.to_datetime(self.incidents['Date'])
        self.incidents['Year']=pd.DatetimeIndex(self.incidents['Date']).year
        self.incidents=self.incidents[['Location','Year','Longitude','Latitude','Count']]
        self.incidents=self.incidents.loc[self.incidents['Year']==2018]
        self.incidents=self.incidents.groupby('Location').sum().reset_index()
        self.incidents['Longitude']=self.incidents['Longitude']/self.incidents['Count']
        self.incidents['Latitude']=self.incidents['Latitude']/self.incidents['Count']
        self.incidents['Year']=self.incidents['Year']/self.incidents['Count']
    def Signal_Handling(self):
        self.signals=pd.read_csv('Traffic_Signals.csv')
        self.signals.rename(columns={'INSTDATE':'Date','INT_TYPE':'Signal Type','latitude':'Latitude',\
                                     'longitude':'Longitude'},inplace=True)
        self.signals['Intersection']=self.signals['FIRSTROAD']+' & '+self.signals['SECONDROAD']
        self.signals['Date']=pd.to_datetime(self.signals['Date'])
        self.signals['Year']=pd.DatetimeIndex(self.signals['Date']).year
        self.signals['Signal Type']=self.signals['Signal Type'].astype(str)
        self.signals=self.signals[['Intersection','Signal Type','Year','Longitude','Latitude','Count']]
        
        self.signals=self.signals.loc[self.signals['Year']<=2018]
        self.signals=self.signals.groupby('Intersection').sum().reset_index()
        
        self.signals['Longitude']=self.signals['Longitude']/self.signals['Count']
        self.signals['Latitude']=self.signals['Latitude']/self.signals['Count'] 
        self.signals['Year']=self.signals['Year']/self.signals['Count'] 
        
    def Sign_Handling(self):
        self.signs=pd.read_csv('Traffic_Signs.csv')
        self.signs.rename(columns={'BLADE_TYPE':'Type','INSTDATE':'Date'},inplace=True)
        
        self.signs['Date']=pd.to_datetime(self.signs['Date'])
        self.signs['Year']=pd.DatetimeIndex(self.signs['Date']).year
        self.signs['Count']=1
        self.signs=self.signs[['Type','Year','POINT','Count']]
        self.signs=self.signs.groupby('POINT').sum().reset_index()
        self.signs['Year']=self.signs['Year']/self.signs['Count']
        self.signs=self.signs.loc[self.signs['Year']<=2018]
        coord=self.signs['POINT'].str.split(expand=True)
#         
        for i in range(len(coord.columns)):
            
            if (i>0):
                coord[i]= coord[i].str.strip(',()')

        coord.drop(columns=0,inplace=True)
        self.signs.drop(columns='POINT',inplace=True)
        self.signs=pd.merge(self.signs,coord,left_index=True,right_index=True)

    def Volume_Handling(self):
        self.volume=pd.read_csv('Traffic_Volumes_for_2018.csv')
        self.volume.rename(columns={'YEAR':'Year','SECNAME':'Location','VOLUME':'Volume'},inplace=True)
        coord=self.volume['multilinestring'].str.split(expand=True)
        for i in range(len(coord.columns)):
            
            if (i>0):
                coord[i]= coord[i].str.strip(',()')
        coord.drop(columns=0,inplace=True)
        self.volume=self.volume[['Year','Location','Volume']]
        self.volume=pd.merge(self.volume,coord,left_index=True,right_index=True)
    
    def get_Speed(self):
        return self.speed_limit
    def get_Camera(self):
        return self.cameras
    def get_Incident(self):
        return self.incidents
    def get_Signal (self):
        return self.signals
    def get_Sign (self):
        return self.signs
    def get_Volume(self):
        return self.volume
        

**Weather data Class**

In [39]:
class Weather_Features():
#     def __init__(self):
#         self.download_weather_data()
        
    def download_weather_data(self):
        self.url_template = "https://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID={station}&Year={year}&Month={month}&Day=14&timeframe={timeframe}&submit=Download+Data"
        self.url = self.url_template.format(station = 50430, year=2018, month=1, timeframe = 1)
        self.weather_data = pd.read_csv(self.url, index_col='Date/Time', parse_dates=True)
        self.weather_data.columns = [col.replace('\xb0', '') for col in self.weather_data.columns]
        self.data_df = pd.DataFrame(self.weather_data)
        self.data_df = self.data_df[['Temp (C)','Visibility (km)']].describe().reset_index()
        return self.data_df


**Map Class**

In [41]:
road=Road_Features()
speed=road.get_Speed()
camera=road.get_Camera()
incident=road.get_Incident()
signal=road.get_Signal()
sign=road.get_Sign()
volume=road.get_Volume()
weather=Weather_Features().download_weather_data()


KeyError: 0

In [ ]:
print(volume)

In [ ]:

def download_weather_data(st, y, m=1, d=True):
    """ returns a DataFrame with weather data from climate.weather.gc.ca"""
    url_template = "https://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID={station}&Year={year}&Month={month}&Day=14&timeframe={timeframe}&submit=Download+Data"
    url = url_template.format(station = st, year=y, month=m, timeframe = 2)
    weather_data = pd.read_csv(url, index_col='Date/Time', parse_dates=True)
    weather_data.columns = [col.replace('\xb0', '') for col in weather_data.columns]
    data_df = pd.DataFrame(weather_data)
    
    return data_df

In [ ]:
download_weather_data(2205, 1918, 1, True)

This is a test